In [1]:
#Incluindo libs necessarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from time import time

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn.cluster import SpectralClustering, AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from itertools import combinations

In [2]:
#Importando dados e removendo atributo target
dfData = pd.read_csv("dataset1.csv").drop(columns = "target")

#Exibindo formato dos dados
print(dfData.shape)
print(dfData.columns)
dfData.head()

#Definindo conjunto de atributos
atributos = [ f'V{x}' for x in range(1,14) ]

#Convertendo tudo para float para remover warnings
def int_to_float(c):
    if isinstance(c, float):
        return c
    else:
        try:
            return float(c)
        except:
            return None

for atributo in atributos:
    dfData[atributo] = dfData[atributo].apply(int_to_float)



(506, 13)
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13'],
      dtype='object')


In [3]:
#Plotando dados
sns.pairplot(dfData)
plt.show()

KeyboardInterrupt: 

## KMeans

In [ ]:
np.random.seed(1001001)


# Inicialização de variaveis
sample_size=150
best_sil = -999
data = dfData

for init in ['k-means++', 'random']:
    for k in range(3,8):
        estimator = KMeans(init=init, n_clusters=k, n_init=10, max_iter=100) # Define o modelo de clustering
        #Rodando kMeans com todas as permutacoes de features possiveis para necontrar a melhor
        for i in range(1,len(atributos)+1):
            for comb in combinations(atributos,i):
                newDf = dfData.filter(comb,axis=1)  # copiando apenas os atributos que serão usados
                data = scale(newDf)
                estimator.fit(data)# execução do fit para fazer a predição
                data = scale(dfData)
                sil = metrics.silhouette_score(data, estimator.labels_,
                                               metric="euclidean",
                                               sample_size=sample_size) #calculo da silhuette para avaliar a predição feita
                if sil > best_sil: #comparação da silhoette atual com a melhor silhouette
                    # Definição dos melhores dados até o momento
                    bestEstimator = estimator
                    best_sil = sil
                    bestComb = comb
                    best_data = pd.DataFrame(data)
                    
#Exibindo resultados
print('better estimator silhouette: %0.4f (%d clusters, %s)' % (best_sil, bestEstimator.n_clusters, bestEstimator.init))
print(bestComb)
dfData['label'] = bestEstimator.labels_
dfData['label'] = dfData['label'].apply(int_to_float)
best_data['label'] = bestEstimator.labels_

In [ ]:
#Plot 2D de cada atributo colorido por cluster
sns.pairplot(data=dfData, vars=bestComb, hue='label')
plt.show()

### Analize inicial:

Pode-se notar que certas variaveis tem graficos melhores distribuidos para uma uma possivel classificação. Se olharmos para as variaveis que foram apontadas pelo algoritmo acima (Vx com Vx é o histograma da variavel):

- V2: 
    Apesar de esperarmos que fosse ter um impacto menos significativo, essa variavel foi apontada com um valor alto de relevancia e mesmo olhando o grafico não conseguirmos visualizar o motivo em particular, uma vez que só conseguimos ver o label verde. Talvez conseguir afirmar que esse valor seja uma label seja bom o suficiente.
    
- V4:
    Essa variavel tem um uso bom para distinguir entre os labels verde e laranja (um pouco para o azul, mas muito especifico), e provavelmente foi selecionada por essa caracteristica.

- V7:
    Notamos o padrão do V2 se repetindo, o grafico mostra em grande parte apenas o label laranja, mas achamos que conseguir afirmar isso já é muito bom para uma variavel ser util.

- V8:
    Essa variavel tem um grafico mais distinto, o label verde se destaca bastante e vimos que isso muitas vezes que isso pode  ser o suficiente para uma variavel ser escolhida.

- V9:
    Pode ser observado que tem um comportamento similar ao da variavel V4 e pode ser muito bom para distinguir entre os tres tipos, de certa forma, "é uma versão melhor da V4".

- V11:
    Notamos o padrão do V2 e V7 se repetindo, acredito que não tem muito o que adicionar.

- V13:
    Segue os mesmos padrões de V4 e V9, acredito que não tem muito o que adicionar.



### PCA:
Usando o PCA foi gerado duas visualizações dos clusters, uma 2D e outra 3D. Pode-se ver que o cluster amarelo é o que mais se separou dos outros, porém ainda dá para se ver uma diferença entre o azul e roxo, isso independente de qual dos graficos se olhe. 

In [ ]:
#Redução de dimencionalidade
pca = PCA(n_components=2)

#Separando as features que iremos usar
x = dfData.loc[:, bestComb].values

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['c1', 'c2'])

#Juntando as 'melhores' features com suas labels
finalDf = pd.concat([principalDf, dfData[['label']]], axis = 1)

#Plot 3D dos atributos por cluster
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(finalDf['c1'], finalDf['c2'], c = finalDf['label'], marker = 'o')
plt.show()

In [ ]:
#Redução de dimencionalidade
pca = PCA(n_components=3)

#Separando as features que iremos usar
x = dfData.loc[:, bestComb].values

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['c1', 'c2', 'c3'])

#Juntando as 'melhores' features com suas labels
finalDf = pd.concat([principalDf, dfData[['label']]], axis = 1)

#Plot 3D dos atributos por cluster
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
fig = plt.figure()
ax1 = fig.add_subplot(111, projection = '3d')
ax1.scatter(finalDf['c1'], finalDf['c2'], finalDf['c3'], c = finalDf['label'], marker = 'o')
plt.show()


### Comparação:
A partir da visualação 2D dos dados, utilizando das variaveis que melhor distinguem (visualmente) um ou mais clusters e vejo se o resultado fica mais claro.

In [ ]:
#Plot 3D dos atributos por cluster
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
fig = plt.figure()
ax1 = fig.add_subplot(111, projection = '3d')
ax1.scatter(best_data[6], best_data[8], best_data[10], c = best_data.label, marker = 'o')
ax1.set(xlabel='V7', ylabel='V9', zlabel='V11')
plt.show()


#### V9:
Seguindo para uma análize mais profunda, podemos ver no gráfico (V7, V9, V11) que os cluster se separam mais em função de V9. Como foi visto no grafico de V9, ele separava fortemente dois cluster (nesse caso sendo eles o cluster amarelo e azul). O roxo por sua vez é o cluster que n conseguia ser bem separado.

In [ ]:
sns.pairplot(data=dfData, vars=['V9'], hue='label')
plt.show()

#### V11:
A variavel V1 aparentemente, dado apenas essas 3 variaveis n tem um impacto muito forte. 

In [ ]:
sns.pairplot(data=dfData, vars=['V11'], hue='label')
plt.show()

#### V7:
V7 por sua vez mostra que valores mais elevados tendem a ser do cluster amarelo. Olhando para o grafico de V7 abaixo, pode notar que ele consegue distinguir um cluster dos outros fortemente e isso, logicamente, se refletiu no grafico 3D.

In [ ]:
sns.pairplot(data=dfData, vars=['V7'], hue='label')
plt.show()

### Analize graficos 3D:

Com esses graficos é possivel ver que o cluster amarelo continua bem separado dos outros dois. Porem ele joga informação fora uma vez que não leva em conta os outros valores. Esse novo grafico ainda não resolvou a existencia de pontos "dentro de outro cluster". Com PDA, em um caso com mais pontos, provavelmente seria mais confiavel.

## Affinity Propagation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering, AffinityPropagation
from sklearn.neighbors import kneighbors_graph
from itertools import combinations

In [ ]:
dfData = pd.read_csv("dataset1.csv").drop(columns = "target")
print(dfData.shape)
print(dfData.columns)
dfData.head()

atributos = [ f'V{x}' for x in range(1,14) ]

def int_to_float(c):
    if isinstance(c, float):
        return c
    else:
        try:
            return float(c)
        except:
            return None

for atributo in atributos:
    dfData[atributo] = dfData[atributo].apply(int_to_float)

In [ ]:
# Inicialização de variaveis
best_sil = -999
data = dfData

for k in range(2,3):
    model = AffinityPropagation()  # Define o modelo de clustering
    for i in range(1,14):
        for comb in combinations(atributos,i):
            newDf = dfData.filter(comb,axis=1)  # copiando apenas os atributos que serão usados
            data = scale(newDf)
            labels = model.fit(data) # execução do fit para fazer a predição
            data = scale(dfData)
            sil = metrics.silhouette_score(data, model.labels_,
                                               metric="euclidean")  #calculo da silhuette para avaliar a predição feita
            
            if sil > best_sil: #comparação da silhoette atual com a melhor silhouette
                # Definição dos melhores dados até o momento
                bestEstimator = model
                bestLabel = labels
                best_sil = sil
                bestComb = comb     
                best_data = pd.DataFrame(data)
                

#Exibindo resultados
print('better estimator silhouette: %0.4f (%d clusters)' % (best_sil, len(bestLabel.cluster_centers_)))
print(bestComb)

dfData['label'] = bestEstimator.labels_
dfData['label'] = dfData['label'].apply(int_to_float)
best_data['label'] = bestEstimator.labels_

### Analise inicial:
Podemos ver que o valor de silhueta é baixo e ele tem um numero de clusters bem alto comparando com o numero de variaveis. Apenas V13 não foi selecionado por esse algoritmo.

### PCA:

Aplicamos PCA para melhorar a visualação dos 29 clusters. Teremos uma 3D e outra 2D.

In [ ]:
#Redução de dimencionalidade
pca = PCA(n_components=3)

#Separando as features que iremos usar
x = dfData.loc[:, bestComb].values

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['c1', 'c2', 'c3'])

#Juntando as 'melhores' features com suas labels
finalDf = pd.concat([principalDf, dfData[['label']]], axis = 1)

#Plot 3D dos atributos por cluster
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
fig = plt.figure()
ax1 = fig.add_subplot(111, projection = '3d')
ax1.scatter(finalDf['c1'], finalDf['c2'], finalDf['c3'], c = finalDf['label'], marker = 'o')
plt.show()


In [ ]:
#Redução de dimencionalidade
pca = PCA(n_components=2)

#Separando as features que iremos usar
x = dfData.loc[:, bestComb].values

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['c1', 'c2'])

#Juntando as 'melhores' features com suas labels
finalDf = pd.concat([principalDf, dfData[['label']]], axis = 1)

#Plot 3D dos atributos por cluster
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(finalDf['c1'], finalDf['c2'], c = finalDf['label'], marker = 'o')
plt.show()

Tentaremos fazer o mesmo que foi feito para visualizar os graficos para montar um induvidualmente.

In [ ]:
#Plot 2D de cada atributo colorido por cluster
sns.pairplot(data=dfData, vars=bestComb, hue='label')
plt.show()

Olhando para cada histograma (Vx com Vx) e grafico, não conseguimos ver nada muito claramente, uma vez que se trata de 29 clusters é muito dificil escolher algo com apenas a visualização dos graficos

### Análize do modelo:

   Dá para ver alguma tendencia no grafico 3D gerado pelo PCA mas muitos clusters estão simplesmente muitos juntos para conseguir fazer uma distinção clara, acreditamos que seja isso a causa de um valor baixo de silhueta.   
   Além disso, achamos curioso o fato do histograma de V2 (V2xV2) que está vazio ser escolhida para ser uma das variaveis no caso com a melhor silhueta.
   Uma coisa que observamos nos modelos 3D é que parece haver uma separação grande entre dois grupos de dados, mas mesmo assim, parecem haver pontos que fazem parte dos dois grupos pertencentes ao mesmo cluster, contudo nós não pudemos ter certeza disso observando os gráficos, pois podem ser apenas tons parecidos de cores 
   Também tentamos hipotetizar sobre o motivo da variável V13 ter sido a unica a ser excluida pelo algoritmo para a definição do melhor caso, porém após analizarmos o plot de todos os datos feito no início do trabalho não pudemos tirar uma conclusão mais definitiva tirando o fato de que na maioria dos pairplots feitos com essa variavel os dados de uma forma geral pareciam estar muito agrupados.

## Aglomerative Clustering

In [ ]:
# Inicialização de variaveis
best_sil = -999
data = dfData

for k in range(2,4):
    # Define o modelo de clustering 
    model = AgglomerativeClustering(n_clusters=k, affinity='euclidean', compute_full_tree='auto', linkage='ward')
    for i in range(1,14):
        for comb in combinations(atributos,i):
            newDf = dfData.filter(comb,axis=1)  # copiando apenas os atributos que serão usados
            data = scale(newDf)
            labels = model.fit_predict(data, 0)# execução do fit para fazer a predição
            data = scale(dfData)
            sil = metrics.silhouette_score(data, model.labels_,
                                               metric="euclidean")#calculo da silhuette para avaliar a predição feita
            
            if sil > best_sil: #comparação da silhoette atual com a melhor silhouette
                # Definição dos melhores dados até o momento
                bestEstimator = model
                best_sil = sil
                bestComb = comb     
                best_data = pd.DataFrame(data)
                


print('better estimator silhouette: %0.4f (%d clusters)' % (best_sil, bestEstimator.n_clusters))
print(bestComb)


dfData['label'] = bestEstimator.labels_
dfData['label'] = dfData['label'].apply(int_to_float)
best_data['label'] = bestEstimator.labels_

### Analise inicial:
O modelo possui apenas dois clusters e uma variavel foi escolida como parametro. Como só uma variavel foi utilizada, não é necessario fazer uma redução de dimencionalidade.

In [ ]:
#Plot 2D de cada atributo colorido por cluster
sns.pairplot(data=dfData, vars=bestComb, hue='label')
plt.show()

### Análize do modelo:

   Como podemos ver, a variavel V7 consegue distinguir muito bem quem é o cluster azul e como só existem 2 clusters, é fácil para esse algoritmo fazer a distinção.
   Acreditamos que a variavel V7 represente bem a distribuição  dos pontos, mesmo ao se considerar os 13 atributos, pois a silhueta que foi obtida foi relativamente elevada ao se considerar que apenas uma variavel foi levada em consideração.

## Análize Geral:

O algoritmo que gerou a melhor silhueta foi o Aglomerative Clustering com uma silhueta de 0,4380, sendo seguido pelo KMeans com a silhueta de 0,4351 e por último veio o Affinity Propagation com 0,2521.
No entanto, a silhueta é apenas uma forma de avaliar o quão próximos um objeto está do seu próprio cluster e distante dos  outros. Além disso, a diferença entre a silhueta do Aglomerative Clustering e a do KMeans foi muito próxima e por conta de o Kmeans usar cinco variaveis em contrapartida ao Aglomerativo que usou apenas uma consideramos que o KMeans pode ser a melhor clusterização obtida.
O Affinity Propagation, por sua vez, teve um resultado muito pior do que os outros dois metodos de clusterização, mas não pudemos definir um motivo sólido para isso além do fato de ele ter criado um número muito maior de clusters em comparação aos outros dois metodos